In [3]:
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import csv

In [36]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

### Crawl links from nhatot.com

In [9]:
# Initialize the array to store the links
arr = []

for page in range(1000):
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        url = f"https://www.nhatot.com/mua-ban-nha-dat?page={page + 1}"
        driver.get(url)

        # Wait for JavaScript to render content
        driver.implicitly_wait(10)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all the links with the specified class
        a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

        # Append the full link to the array
        for a in a_tags:
            full_link = "https://www.nhatot.com" + a['href']
            arr.append(full_link)

        driver.quit()

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('nhatot.csv', index=True)

print("done")

done


## crawl data from each link

In [5]:
frame = pd.read_csv('nhatot1.csv')
frame.head()

,Unnamed: 0,Links
0,0,https://www.nhatot.com/mua-ban-nha-dat-thi-xa-...


## Test lấy dữ liệu

In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Truy cập vào trang web
url = 'https://www.nhatot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/119225036.htm#px=SR-stickyad-[PO-1][PL-top]'
driver.get(url)

# Đợi một chút để trang tải hết nội dung
time.sleep(5)

# Lấy nội dung của trang
page_source = driver.page_source

# Đóng trình duyệt
driver.quit()

# Phân tích cú pháp HTML với BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

price = soup.find("b", class_='pyhk1dv').text
print(price)

address = soup.find_all("span", class_="bwq0cbs")
print(address[4].text)

price_m2 = soup.find("span", itemprop="price_m2").text
rooms = soup.find("span", itemprop="rooms").text
toilets = soup.find("span", itemprop="toilets").text
direction = soup.find("span", itemprop="direction").text
floors = soup.find("span", itemprop="floors").text
property_legal_document = soup.find("span", itemprop="property_legal_document").text
house_type = soup.find("span", itemprop="house_type").text
furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
width = soup.find("span", itemprop="width").text
length = soup.find("span", itemprop="length").text
living_size = soup.find("span", itemprop="living_size").text
size = soup.find("span", itemprop="size").text




3,85 tỷ
Tổ 4, KP Long Khánh 3, Phường Tam Phước, Thành phố Biên Hòa, Đồng Nai
Hướng cửa chính: Tây
36,32 triệu/m²
3 phòng
Tây
3 phòng
3
Đã có sổ
Nhà mặt phố, mặt tiền
Nội thất đầy đủ


## Lấy chi tiết data

In [6]:
# Function to initialize WebDriver
def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to avoid displaying the browser
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Function to extract data from a single page
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(5)  # Wait for page to load
    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, 'html.parser')

    try:
        price = soup.find("b", class_='pyhk1dv').text
    except AttributeError:
        price = None

    try:
        address = soup.find_all("span", class_="bwq0cbs")[4].text
    except (IndexError, AttributeError):
        address = None

    try:
        price_m2 = soup.find("span", itemprop="price_m2").text
    except AttributeError:
        price_m2 = None

    try:
        rooms = soup.find("span", itemprop="rooms").text
    except AttributeError:
        rooms = None

    try:
        toilets = soup.find("span", itemprop="toilets").text
    except AttributeError:
        toilets = None

    try:
        direction = soup.find("span", itemprop="direction").text
    except AttributeError:
        direction = None

    try:
        floors = soup.find("span", itemprop="floors").text
    except AttributeError:
        floors = None

    try:
        property_legal_document = soup.find("span", itemprop="property_legal_document").text
    except AttributeError:
        property_legal_document = None

    try:
        house_type = soup.find("span", itemprop="house_type").text
    except AttributeError:
        house_type = None

    try:
        furnishing_sell = soup.find("span", itemprop="furnishing_sell").text
    except AttributeError:
        furnishing_sell = None

    try:
        width = soup.find("span", itemprop="width").text
    except AttributeError:
        width = None

    try:
        length = soup.find("span", itemprop="length").text
    except AttributeError:
        length = None

    try:
        living_size = soup.find("span", itemprop="living_size").text
    except AttributeError:
        living_size = None

    try:
        size = soup.find("span", itemprop="size").text
    except AttributeError:
        size = None

    return {
        "price": price,
        "address": address,
        "price_m2": price_m2,
        "rooms": rooms,
        "toilets": toilets,
        "direction": direction,
        "floors": floors,
        "property_legal_document": property_legal_document,
        "house_type": house_type,
        "furnishing_sell": furnishing_sell,
        "width": width,
        "length": length,
        "living_size": living_size,
        "size": size
    }

# Function to save data to CSV
def save_to_csv(data_list, filename='raw_data.csv'):
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    all_data = []
    for url in frame['Links']:
        data = extract_data(url)
        all_data.append(data)
        print(f"Processed URL: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()


Processed URL: https://www.nhatot.com/mua-ban-nha-dat-thi-xa-ben-cat-binh-duong/114120310.htm#px=SR-stickyad-[PO-1][PL-top]
